In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Flatten, BatchNormalization, Dense, Dropout, Conv2D, MaxPooling2D

In [ ]:
TRAIN_DIR = './cifar-10/train'

In [ ]:
filenames = os.listdir(TRAIN_DIR)

In [ ]:
labels_df = pd.read_csv('./cifar-10/trainLabels.csv')

In [ ]:
labels_df.head()

In [ ]:
label_encoder = LabelEncoder()
labels_df['label_encoded'] = label_encoder.fit_transform(labels_df['label'])

In [ ]:
id_list = labels_df['id']

In [ ]:
print(id_list[:5])

In [ ]:
from PIL import Image

data = []

for id in id_list:
    image = Image.open(os.path.join(TRAIN_DIR, f'{id}.png'))
    image = np.array(image)
    data.append(image)

In [ ]:
print(len(data))
print(type(data[0]))

In [ ]:
data[0].shape

In [ ]:
labels_df.head()

In [ ]:
X = np.array(data)
y = np.array(labels_df['label_encoded'])

In [ ]:
print(X.shape, y.shape)

In [ ]:
X_scaled = X/255.

In [ ]:
model = keras.Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),

    Flatten(),

    Dense(512, activation='relu'),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics='accuracy'
)

In [ ]:
model.fit(X_scaled, y, validation_split=0.1, epochs=10, verbose=1)

In [ ]:
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(32,32,3))

for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model = keras.Sequential([
    base_model,
    Flatten(),
    BatchNormalization(),

    Dense(128, activation='relu'),
    Dropout(0.2),
    BatchNormalization(),

    Dense(64, activation='relu'),
    Dropout(0.2),
    BatchNormalization(),

    Dense(10, activation='softmax')
])

In [ ]:
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics='accuracy'
)

In [ ]:
history = model.fit(X_scaled, y, validation_split=0.1, epochs=10, verbose=1)